通常情况下，我们一般都会使用交叉验证来作为评估模型的标准，来选择我们最后的模型。但是在一些数据挖掘竞赛中，数据集一般分为训练集和测试集，国内比赛可能根据比赛阶段划分多个测试集，由于数据集采样和分布的原因导致训练集和线上测试集可能存在分布不一致的情况，这时候交叉验证（CV）无法准确的评估模型在测试集上的效果，导致线上线下不统一，分数上不去。而缓解这一问题的黑科技，就是对抗验证。

对抗验证：用来验证训练集和测试集数据分布是否一致;从训练集中找出一部分跟测试集类似的数据作为验证集再进行训练，最后再用训练好的模型用测试集测试。从而得到更好的结果。

In [40]:
#验证训练集和测试集数据分布是否一致

import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

iris_dataset = load_iris()  #加载鸢尾花数据
X_train, X_test, y_train, y_test = train_test_split(iris_dataset['data'], iris_dataset['target'], random_state=0)  #拆分数据

# 开始给训练集和测试集打上标签
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train['target'] = 0
X_test['target'] = 1

#将训练集和测试集合并
data = pd.concat([X_train, X_test], axis=0)
data = data.reset_index(drop=True)

#取出label
label = data['target']
data.drop(['target'], axis=1, inplace=True)

#开始验证训练集和测试集是否一样，如果auc越接近0.5，就说明数据分布一样
X, y = data, pd.DataFrame(label) 
skf = StratifiedKFold(n_splits=3) #分层交叉验证

auc = []
for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    x_train, x_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index, :], y.iloc[train_index, :].values.ravel(), y.iloc[test_index, :].values.ravel()
    #使用lightgbm训练
    lgb = LGBMClassifier(learning_rate=0.01, n_estimators=1000, max_depth=5)
    lgb.fit(x_train, y_train)
    y_pred = lgb.predict(x_test)
    #计算AUC的值
    auc_score = roc_auc_score(y_test, y_pred)
    auc.append(auc_score)
print(np.array(auc).mean())

0.5697134381344908


In [36]:
#在训练集上选取和测试集相似的验证集

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train['target'] = 0
X_test['target'] = 1

#将训练集和测试集合并
data = pd.concat([X_train, X_test], axis=0)
data = data.reset_index(drop=True)

#取出label
label = data['target']
data.drop(['target'], axis=1, inplace=True)

#将原始数据进行训练和预测
lgb = LGBMClassifier(learning_rate=0.01, n_estimators=1000, max_depth=5)
lgb.fit(data, label)
test_pre = lgb.predict(data)

#在测试集中增加一列，该列为原始数据训练后再预测的结果
X_train['is_test_prob'] = test_pre[:len(X_train)]

#排序结果按值降序，前面多列为训练集预测为测试集的数据
X_train= X_train.sort_values('is_test_prob',ascending=False).reset_index(drop=True)

#取前20%作为验证集
x_vaild= X_train.iloc[:int(0.2 * len(X_train)), ]
X_vaild = x_vaild.copy()
X_vaild.drop(['is_test_prob'], axis=1, inplace=True)
X_vaild

,0,1,2,3,target
0,5.6,2.5,3.9,1.1,0
1,5.9,3.2,4.8,1.8,0
2,6.3,3.3,4.7,1.6,0
3,5.0,3.3,1.4,0.2,0
4,5.9,3.0,4.2,1.5,0
5,5.0,3.2,1.2,0.2,0
6,5.8,4.0,1.2,0.2,0
7,6.0,3.0,4.8,1.8,0
8,5.2,4.1,1.5,0.1,0
9,6.7,3.1,4.4,1.4,0


In [39]:
#将原始的样本进行拆分得到训练集，验证集，测试集。那么线上模型验证时，在这样的数据上调参得到的模型，如果在验证集上效果稳定，那么应用在测试集上，大概率结果是一致的。
data = pd.concat([X_train, X_vaild], axis=0)
data = data.reset_index(drop=True)

#取出label
label = data['target']
data.drop(['target'], axis=1, inplace=True)

X, y = data, pd.DataFrame(label) 
skf = StratifiedKFold(n_splits=3) #分层交叉验证

res = []
for index, (train_index, valid_index) in enumerate(skf.split(X, y)):
    x_train, x_valid, y_train, y_valid = X.iloc[train_index, :], X.iloc[valid_index, :], y.iloc[train_index, :].values.ravel(), y.iloc[valid_index, :].values.ravel()
    #使用lightgbm训练
    lgb = LGBMClassifier(learning_rate=0.01, n_estimators=1000, max_depth=5)
    lgb.fit(x_train, y_train)
    result = lgb.score(x_valid, y_valid)
    res.append(result)
print("模型在验证集上的精度为：", np.array(res).mean())

模型在验证集上的精度为： 1.0
